## Process Data
- Goal: Reformat autolabelled refusal dataset to be in nicer format (since it was generated using the previous teams repo)

Jsonl explanation:
'inputs': t
'ids': combined id,
'split': split in original anthropic dataset,
'category': category in original anthropic dataset, whether annotator chose this example or other one,
'index': index number,
'scale_label_explanation': explanation given by autograder,
'scale_label_confidence': 1 to 10 where higher is more confident,
'scale_labels': 1 to 10 where lower is bigger rejection and higher is bigger compliance,
'scale_label_model': model used to autograde,
'labels': 'complied', 'refused' or 'ambiguous',
'label_explanation': what method was used to decide label, usually threshold on score

In [9]:
# open jsonl file
import json
datasets = [
    "../data/refusal/anthropic_balanced_apr_23.jsonl",
    "../data/refusal/anthropic_raw_apr_23.jsonl"
]
for dataset in datasets[:1]:
    data_list = []
    with open(dataset, 'r') as file:
        for line in file:
            data = json.loads(line)
            if data["labels"] == "high-stakes":
                data["labels"] = "complied"
            elif data["labels"] == "low-stakes":
                data["labels"] = "refused"
            data_list.append(data)

    # write the modified data back to the file
    with open(dataset, 'w') as file:
        for line in data_list:
            file.write(json.dumps(line) + "\n")